# EDA & Model Development 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import random

## Initial Setup

In [2]:
#https://raw.githubusercontent.com/jchristo12/fantasy_football/master/data/full_data.csv
df = pd.read_csv('https://github.com/jchristo12/fantasy_football/blob/master/data/full_data.csv?raw=true')

In [3]:
df.columns

Index(['pk', 'gid', 'seas', 'wk', 'player', 'fname', 'lname', 'full_name',
       'team', 'pos1',
       ...
       'humd', 'ou', 'sprv', 'ptsv', 'ptsh', 'udog', 'gen_cond', 'udog_binary',
       'def_team', 'f_pts'],
      dtype='object', length=161)

In [4]:
#remove rows that have NaN for the shifted variables
df_clean = df[~df.loc[:,'seas_pa':'seas_tdret'].isna().all(axis=1)]

##### We are only focused on offensive players right now. Therefore, we will specifiy the positions we want to retain in the data and remove everyone else

In [5]:
#store positions we are concerned about; will use these to filter out 
pos_of_interest = ['QB', 'RB', 'WR', 'TE']
#filter out positions we don't care about
df_clean2 = df_clean[df_clean['pos1'].isin(pos_of_interest)]

##### Convert each column to the appropriate data type (i.e. make the categorical data categories)

In [6]:
#set the column types
col_dtypes = {'category': ['seas', 'wk', 'pos1', 'team', 'udog', 'dv', 'v', 'h', 'day', 'cond', 'stad', 'wdir',
                          'surf', 'gen_cond', 'def_team']}
#flip the key and values around so they will work in the argument for 'astype()'
col_dtypes_alt = {old: new for new, old_all in col_dtypes.items() for old in old_all}

In [7]:
df_clean2 = df_clean2.astype(col_dtypes_alt)

##### Segment out the WR data

In [8]:
df_wr = df_clean2.loc[df_clean2['pos1']=='WR', :]

##### Only focus on week 10 data

In [9]:
df_wr10 = df_wr.loc[df_wr['wk'] == 10, :]

## EDA

Only focusing on the WR data right now. Will need to build this so that each action is extendable to the other positions

### Training/Test split

In [12]:
from sklearn.model_selection import train_test_split

In [31]:
#set the random seed for reproducability
random.seed(837)

In [32]:
#break out the data between training and test
train_wr, test_wr = train_test_split(df_wr10, train_size=0.75, test_size=0.25, shuffle=True)

In [33]:
#shape of the data
print(train_wr.shape, test_wr.shape)

(1625, 161) (542, 161)


### Missing Data analysis

#### Basic analysis

In [34]:
#create a series of percent of missing data
missing_data_pct = train_wr.isna().sum() / train_wr.shape[0]

In [35]:
#list all columns with missing data that is greater than 25%
missing_data_pct[missing_data_pct > 0.25].sort_values(ascending=False)

roll_td_to_int           0.992000
seas_td_to_int           0.990154
roll_yds_per_comp        0.985846
seas_yds_per_comp        0.980923
roll_comp_pct            0.972923
seas_comp_pct            0.960000
career_td_to_int         0.857846
career_yds_per_comp      0.846769
career_comp_pct          0.798769
roll_carry_to_td         0.737231
roll_ryds_per_carry      0.736615
roll_avg_ret             0.685538
roll_ret_to_td           0.685538
roll_carry_to_fuml       0.662769
seas_ret_to_td           0.653538
seas_avg_ret             0.653538
seas_carry_to_td         0.633231
seas_ryds_per_carry      0.632615
career_avg_ret           0.600000
career_ret_to_td         0.600000
seas_carry_to_fuml       0.533538
career_ryds_per_carry    0.328615
career_carry_to_td       0.327385
dtype: float64

#### Take action on missing data

##### Drop columns with too many missing values

In [36]:
#drop columns with missing data greater than 25%
#store the column names
missing_cols_del = missing_data_pct[missing_data_pct > 0.25].sort_values(ascending=False).index
#drop the columns and store as new dataframe
train_wr_miss = train_wr.drop(missing_cols_del, axis=1, inplace=False)

In [37]:
#check the shape
train_wr_miss.shape

##### Impute the rest of the missing values

In [39]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

In [ ]:
def tree_impute():
    

In [41]:
train_wr_miss.groupby('player').agg({'pk': 'count'}) # / train_wr_miss.shape[1]

,pk
player,
AA-1100,1
AB-0800,4
AB-1200,2
AB-2000,7
AB-3500,6
AB-3800,4
AC-0100,3
AC-0237,1
AC-1300,3
